In [ ]:
library(Seurat)
library(ggplot2)        
library(openxlsx)

# Load data

In [ ]:
# Rat batch 1
rat1 <- readRDS("/projects/mludwig/DVC/data/rat_snRNA/20220621_brain_0284_seurat_obj.rds")
rat1$pool <- rat1$orig.ident
rat1[["percent.mt"]] <- PercentageFeatureSet(rat1, pattern = "^Mt-")
rat1$run <- "rat1"
rat1 <- subset(rat1, nCount_RNA > 3e+04 | nCount_RNA < 500 | percent.mt  > 2, invert = T)

# Rat batch 2
rat2 <- readRDS("/projects/mludwig/DVC/data/rat_snRNA/20220621_brain_0285_seurat_obj.rds")
rat2$pool <- rat2$orig.ident
rat2[["percent.mt"]] <- PercentageFeatureSet(rat2, pattern = "^Mt-")
rat2$run <- "rat2"
rat2 <- subset(rat2, nCount_RNA > 3e+04 | nCount_RNA < 500 | percent.mt  > 2, invert = T)

# Rat batch 3
rat3 <- readRDS("/projects/mludwig/DVC/data/rat_snRNA/20220801_brainDVC_1_seurat_obj.rds")
rat3$pool <- rat3$orig.ident
rat3[["percent.mt"]] <- PercentageFeatureSet(rat3, pattern = "^Mt-")
rat3$run <- "rat3"
rat3$hash.mcl.ID <- gsub("-", "", rat3$hash.mcl.ID)
rat3 <- subset(rat3, nCount_RNA > 3e+04 | nCount_RNA < 500 | percent.mt  > 2, invert = T)

# Rat batch 4
rat4 <- readRDS("/projects/mludwig/DVC/data/rat_snRNA/20220801_brainDVC_2_seurat_obj.rds")
rat4$pool <- rat4$orig.ident
rat4[["percent.mt"]] <- PercentageFeatureSet(rat4, pattern = "^Mt-")
rat4$run <- "rat4"
rat4$hash.mcl.ID <- gsub("-", "", rat4$hash.mcl.ID)
rat4 <- subset(rat4, nCount_RNA > 3e+04 | nCount_RNA < 500 | percent.mt  > 2, invert = T)

# Rat batch 5
rat5 <- readRDS("/projects/mludwig/DVC/data/rat_snRNA/221025_brain_1_seurat_obj.rds")
rat5$pool <- rat5$orig.ident
rat5[["percent.mt"]] <- PercentageFeatureSet(rat5, pattern = "^Mt-")
rat5$run <- "rat5"
rat5$hash.mcl.ID <- gsub("-", "", rat5$hash.mcl.ID)
rat5 <- subset(rat5, nCount_RNA > 3e+04 | nCount_RNA < 500 | percent.mt  > 2, invert = T)

# Rat batch 6
rat6 <- readRDS("/projects/mludwig/DVC/data/rat_snRNA/221025_brain_2_seurat_obj.rds")
rat6$pool <- rat6$orig.ident
rat6[["percent.mt"]] <- PercentageFeatureSet(rat6, pattern = "^Mt-")
rat6$run <- "rat6"
rat6$hash.mcl.ID <- gsub("-", "", rat6$hash.mcl.ID)
rat6 <- subset(rat6, nCount_RNA > 3e+04 | nCount_RNA < 500 | percent.mt  > 2, invert = T)

# Merge 

In [ ]:
rat.list <- list(rat1, rat2, rat3, rat4, rat5, rat6) 

names(rat.list) <- c("rat1", "rat2", "rat3", "rat4", "rat5", "rat6")

rat <- merge(x = rat.list[[1]], y = rat.list[-1])

rat@meta.data$treatment <- dplyr::case_when(
  rat@meta.data$hash.mcl.ID%in%c(paste0('Animal', c(2,12,13,21,24,25,35,36,40,41))) ~ 'V-A',
  rat@meta.data$hash.mcl.ID%in%c(paste0('Animal', c(3,7,8,10,11,18,22,28,29,37,42))) ~ 'A8-A',  
  rat@meta.data$hash.mcl.ID%in%c(paste0('Animal', c(1,5,6,23,26,27,30,32,33,34,38))) ~ 'A8-C',                                       
  rat@meta.data$hash.mcl.ID%in%c(paste0('Animal', c(4,9,14,15,16,17,20,31,39,43,44))) ~ 'WM-C', 
  rat@meta.data$hash.mcl.ID%in%c(paste0('Animal', c(45,46,47,48,49,50))) ~ 'L-A', 
)

rat$hash.ID <- rat$hash.mcl.ID
rat$hash.ID <- paste0(rat$treatment, "-", gsub("Animal", "", rat$hash.ID))

# Normalize

In [ ]:
rat <- SCTransform(rat, verbose = F, method = "qpoisson")

# Run dimensionality reduction

In [ ]:
# PCA
VariableFeatures(rat[["SCT"]]) <- rownames(rat[["SCT"]]@scale.data)
rat <- RunPCA(rat, verbose = F, npcs = 100)

# Select number of PCs
ElbowPlot(rat, ndims = 100)

# UMAP
rat <- RunUMAP(rat, dims = 1:30, n.neighbors = 50)
DimPlot(rat, group.by = "run", shuffle = T)

# Find clusters

In [ ]:
rat <- FindNeighbors(rat, dims = 1:30, k.param = 50, verbose = F)
rat <- FindClusters(rat, resolution = 0.1, verbose = F)
rat <- FindClusters(rat, resolution = 1, verbose = F)

# Save

In [ ]:
rat@meta.data  <- rat@meta.data[, c("nCount_RNA", "nFeature_RNA", 
                                    "nCount_SCT", "nFeature_SCT",
                                    "percent.mt", "pool", "hash.ID", 
                                    "treatment", "run", 
                                    "SCT_snn_res.0.1", "SCT_snn_res.1")]

saveRDS(rat, file = "/projects/mludwig/DVC/output/Seurat_objs/rat/rat_Seurat_obj.rds")